In [ ]:
CATALOGO_DESTINO = "spotify_analytics"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_track_artists"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

### Configuração

Define catálogo, schema e nome da tabela bridge de tracks e artistas.

### DDL - Tabela Bridge Track-Artists

Resolve relacionamento N:N entre tracks e artistas:
- Uma track pode ter múltiplos artistas (feat.)
- Um artista pode ter múltiplas tracks

Campos:
- **track_id** (FK): Referência para tb_tracks
- **artist_id** (FK): Identificador do artista no Spotify
- **artist_name**: Nome do artista (desnormalizado para performance)
- Colunas de metadata

Chave composta: (track_id, artist_id)

In [ ]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    track_id STRING NOT NULL COMMENT 'Chave estrangeira para tb_tracks.track_id',
    artist_id STRING NOT NULL COMMENT 'Identificador único do artista no Spotify',
    artist_name STRING COMMENT 'Nome do artista (desnormalizado para queries)',
    dt_ingestion TIMESTAMP COMMENT 'Data/hora de ingestão no lake',
    dc_origem STRING COMMENT 'Origem dos dados',
    CONSTRAINT pk_track_artists PRIMARY KEY (track_id, artist_id)
)
USING DELTA
COMMENT 'Tabela bridge: resolve relacionamento N:N entre tracks e artistas'
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true',
    'quality' = 'silver',
    'grain' = 'one_row_per_track_artist_combination'
);
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")